In [2]:
from scipy.signal import lfilter, lfilter_zi
import scipy.signal as ss
import numpy as np
import pyaudio as pa
from scipy.io import wavfile
from time import time
from IPython.display import Audio
import matplotlib.pyplot as plt
import librosa
import librosa.display as ld

In [3]:
def tofloat32(x): return (x / np.abs(x).max()).astype(np.float32)
p = pa.PyAudio()

In [4]:
fs, data = wavfile.read('Feliz_cumple.wav')
data = tofloat32(data)
print(len(data))
print(fs)

1068390
65536


In [5]:
def transfer_function(fs, tau, T60, filter_type):
    N = int(np.round(tau*fs, 0))
    g = 10**(-3*tau/T60)
    b = np.zeros(N + 1)
    a = np.zeros(N+1)
    
    if filter_type == 'comb_filter':
        b[0] = 1.0
    else:
        b[0] = -g
        b[N] = 1.0
    a[0] = 1.0
    a[N] = -g
    return np.poly1d(b), np.poly1d(a)


def comb_filter(fs : int, data, tau : float, T60 : float, zi = None) -> tuple[np.array, np.array]:
    N = int(np.round(tau * fs, 0))
    a = np.zeros(N + 1)
    a[[0, N]] = [1, 10 ** (-3 * tau / T60)]
    b = [1.0]
    if zi is None: zi = np.zeros(max(len(b), a.size) - 1)
    print(len(b), a.size)    
    return lfilter(b, a, data, zi = zi)

In [29]:
def create_tf(fs, tau, T60, filter_type):
    
    h = []
    for i, t in enumerate(tau):
        num, den = transfer_function(fs, t, T60[i], filter_type)
        h.append([num.coef, den.coef])
        
    return h
    
    if filter_type == 'comb_filter':
        
        num, denom = 0, 1
        for i, transfer in enumerate(h):
            denom *= transfer[1] 
            temp_num = transfer[0]
            
            for j, other_div in enumerate(h):
                if j == i: continue
                temp_num *= other_div[1]
                
            num += temp_num
        
    elif filter_type == 'ap_filter':
        num, denom = h[0][0] * h[1][0], h[0][1] * h[1][1]
        
    return num.coef, denom.coef

In [48]:
chunksize = 1024 * 2
tau_comb = np.array([50e-3, 21e-3, 33e-3, 47e-3])#[99e-3, 88e-3, 103e-3, 105e-3][9.1e-3, 7.7e-3, 10.1e-3, 11.2e-3]
g_comb = np.array([0.81, 0.86, 0.79, 0.73])
T60_comb = [0.2, 0.1, 0.1, 0.1]#-3*tau_comb / np.log10(g_comb)
tau_ap = np.array([100e-3, 100e-3/3])
g_ap = 0.7
T60_ap = -3*tau_ap/np.log10(g_ap)
y = data

In [49]:
stream = p.open(channels = 1, format = pa.paFloat32, rate = fs, output = True)
stream.start_stream()


y_chunk = y[ : chunksize]

h_comb = create_tf(fs, tau_comb, T60_comb, 'comb_filter')
h_ap = create_tf(fs, tau_ap, T60_ap, 'ap_filter') # h_ap = [1, 1]

def apply(y, zi):
    out = np.zeros(y.size)
    
    for i, filt in enumerate(h_comb):
        f, z = lfilter(*filt, y, zi = zi[i])
        out += f
        zi[i] = z
    
    for i, filt in enumerate(h_ap):
        out, z = lfilter(*filt, out, zi = zi[i + len(h_comb)])
        zi[i + len(h_comb)] = z
    
    return out
   
zi = []

for transfer in h_comb:
    zi.append(np.zeros(-1 + max(len(transfer[0]), len(transfer[1]))))
for transfer in h_ap: 
    zi.append(np.zeros(-1 + max(len(transfer[0]), len(transfer[1]))))

y_filt = apply(y_chunk, zi = zi)
i = 1

norm = np.abs(y).max()

while len(y_filt) > 0 and i < 250:
    stream.write((y_filt / (6*norm)).astype(np.float32).tobytes())
    y_chunk = y[i * chunksize : (i + 1) * chunksize]
    y_filt = apply(y_chunk, zi = zi) 
    i += 1
    
stream.stop_stream()
stream.close()
#Audio(res, rate = fs)

KeyboardInterrupt: 

In [27]:
h_tot[0].size

18636

In [28]:
h_tot[1].size

18636

In [43]:
[(k[0].size, k[1].size) for k in h_ap]

[(6555, 6555), (2186, 2186)]

In [44]:
[(k[0].size, k[1].size) for k in h_comb]

[(3278, 3278), (1377, 1377), (2164, 2164), (3081, 3081)]